# Práctica 2: Aprendizaje y selección de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

### Alumnado:

* Alejandro Gómez Escribano
* Mykola Mandzyak

\* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo

En esta práctica estudiaremos los modelos más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los clasificadores resultantes. Además, veremos métodos de selección de modelos orientados a obtener una configuración óptima de hiperparámetros.

Los conjuntos de datos utilizados son `Diabetes` y `Wisconsin`.

# 1. Preliminares

Antes de comenzar cargamos las librerías para que estén disponibles posteriormente:

In [ ]:
# Third party
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from imblearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from imblearn import FunctionSampler
from sklearn.ensemble import IsolationForest
import pandas as pd

# Local application
import miner_a_de_datos_aprendizaje_modelos_utilidad as utils

Ahora debemos fijar una semilla para que los experimentos sean reproducibles:

In [ ]:
random_state = 18453 #cambiarlo en el imputador de outliers del archivo utils

## 1.1 Modelos e Hiperparámteros tenidos en cuenta 

Los iniciaremos con sus valores por defecto o los más básicos posibles para poder compararlos después.

### 1.1.1 Vecinos más cercanos

* `n_neighbors`: Número de vecinos más cercanos (por defecto, `n_neighbors=5`). Es importante señalar que aumentar el número de vecinos más cercanos provoca que el clasificador sea más robusto al ruido, pero al mismo tiempo se obtienen fronteras de decisión menos distinguidas.

* `weights`: Función de ponderación de los vecinos más cercanos (por defecto, `weights="uniform"`). Si consideramos que todos los vecinos cercanos tienen la misma importancia utilizaremos`weights="uniform"`, por el contrario, si consideramos que los vecinos más cercanos tienen más importancia y deben contribuir más a la regresión que los puntos lejanos, utilizaremos `weights="distance"`.

* `metric`: Distancia métrica que utilizará el árbol (por defecto, `metric="minkowski"`, pero como el hiperparámetro `p=2` equivale a usa una distancia euclídea), las distancias a considerar serán `metric="euclidean"` y `metric="manhattan"`.

In [ ]:
n_neighbors = 10

k_neighbors_model = KNeighborsClassifier(n_neighbors)

### 1.1.2 Árboles de decisión

* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="gini"`).
* `max_depth`: Altura máxima del árbol de decisión (por defecto, `max_depth=None`).
* `ccp_alpha`: Parámetro de complejidad usado para el algoritmo de post-poda *Minimal Cost-Complexity Pruning* (por defecto, `ccp_alpha=0.0`).
* `splitter`: La estrategia utilizada para elegir la división en cada nodo. Las estrategias admitidas son "best" para elegir la mejor división y "random" para elegir la mejor división aleatoria (por defecto, `splitter=best`).


In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

### 1.1.3 Adaptative Boosting (AdaBoost)

* `base_estimator`: Estimador base que usa el ensemble (por defecto, `base_estimator=None`). Si no se especifica, se utiliza un árbol de decisión de profundidad uno.
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=50`).
* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=1.0`).
* `ccp_alpha`: Parámetro de complejidad usado para el algoritmo de post-poda *Minimal Cost-Complexity Pruning* (por defecto, `ccp_alpha=0.0`).

In [ ]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

### 1.1.4 Bootstrap Aggregating (Bagging)

* `base_estimator`: Estimador base que usa el ensemble (por defecto, `base_estimator=None`). Si no se especifica, se utiliza un árbol de decisión sin podar.
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=10`).
* `bootstrap`: Cuándo el muestreo de instancias es con reemplazo (por defecto, `bootstrap=True`).
* `bootstrap_features`: Cuándo el muestreo de características es con reemplazo (por defecto, `bootstrap_features=False`).
* `random_state`: Semilla para controlar la reproducibilidad de los experimentos (por defecto, `random_state=None`).

In [ ]:
bagging_model = BaggingClassifier(random_state=random_state)

### 1.1.5 Random Forests

* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=100`).
* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="gini"`).
* `max_features`: Número de características a considerar en cada nodo del árbol (por defecto, `max_features="auto"`).


In [ ]:
random_forest_model = RandomForestClassifier(random_state=random_state)

### 1.1.6 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=0.1`).
* `n_estimators`: Número de estimadores a aprender (por defecto, `n_estimators=100`).
* `criterion`: Criterio utilizado para medir la calidad de una partición (por defecto, `criterion="friedman_mse"`).
* `max_depth`: Altura máxima de los árboles de decisión (por defecto, `max_depth=3`).
* `ccp_alpha`: Parámetro de complejidad usado para el algoritmo de post-poda *Minimal Cost-Complexity Pruning* (por defecto, `ccp_alpha=0.0`).

### 1.1.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

* `learning_rate`: Parámetro de regularización para controlar la contribución de cada estimador (por defecto, `learning_rate=0.1`).
* `max_iter`: Número máximo de iteraciones del algoritmo (por defecto, `max_iter=100`).
* `max_leaf_nodes`: Número máximo de nodos hoja (por defecto, `max_leaf_nodes=31`).

In [ ]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

## 1.2 Términos importantes

### 1.2.1 Hiperparámetro

Parámetro del algoritmo de aprendizaje que modifica como se induce el modelo.

### 1.2.2 Sesgo

Diferencia entre el valor esperado y el valor obtenido.

Se debe a las suposiciones incorrectas del modelo.

Este error suele reducirse usando modelos más complejos o una mayor dimensionalidad de los datos.

### 1.2.3 Varianza

Variabilidad de la predicción esperada y la
predicción obtenida.

Se produce cuando el modelo aprende patrones espúreos debido al ruido de los datos. 

Este error suele reducirse aumentando el número de ejemplos del conjunto de datos de entrenamiento o usando modelos más simples.

![](https://bookdown.org/content/2031/images/sesgo_varianza.png)

Tendremos modelos que tengan alto sesgo o varianza, por lo que deberemos tratar estos problemas dependiendo del error de cada modelo.

## 1.3 Evaluación de modelos

Para la evaluación de modelos, utilizaremos una validación cruzada estratificada para asegurarnos buenas estimaciones y evitar así el sesgo y la varianza. Utilizaremos 10 carpetas y 5 repeticiones:

In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

Además como criterio de evaluación utilizaremos la media entre `Accuracy`, `Recall` y `ROC`, aunque utilizamos `ROC`, consideramos `Recall` a parte para ponderar más los aciertos en positivos ya estamos ante conjuntos de datos que tratan efermedades y hemos hecho esta consideración.

Aplicaremos esto a los conjuntos de datos utilizados en la práctica anterior, comenzando por `Diabetes`.

# 2. Diabetes

## 2.1 Carga de datos

Cargamos datos y comprobamos que se han cargado correctamente

In [ ]:
filepath= "../input/pima-indians-diabetes-database/diabetes.csv"
target = "Outcome"

data = utils.load_data(filepath,None,target)
data.index=range(data.shape[0])
data.sample(8,random_state=random_state)

A continuación, obtendremos nuestras variables predictoras por un lado y la variable objetivo por otro.

In [ ]:
target = "Outcome"

(X, y) = utils.divide_dataset(data, target)

Ahora comprobamos que se han separado correctamente:

**Variables Predictoras**

In [ ]:
X.sample(5, random_state=random_state)

**Variable Clase**

In [ ]:
y.sample(5, random_state=random_state)

Ahora, divididiremos el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado (70% train y 30% test):

In [ ]:
stratify = y
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Ahora comprobamos que hemos separado correctamente el conjunto en train y test:

**Train - Variables Predictoras**

In [ ]:
X_train.sample(5, random_state=random_state)

**Train - Variable Clase**

In [ ]:
y_train.sample(5, random_state=random_state)

**Test - Variables Predictoras**

In [ ]:
X_test.sample(5, random_state=random_state)

**Test - Variable Clase**

In [ ]:
y_test.sample(5, random_state=random_state)

In [ ]:
#Establecemos la 'X' y la 'y' de esta forma por comodidad para los modelos

X = X_train
y = y_train

**Parámetros del pipeline**

In [ ]:
variables =['BMI','Glucose','BloodPressure']
imputar = utils.imputador(variables)

lista = ['Insulin','SkinThickness']
eliminar_columnas = utils.dropVar(lista)

eliminar_outliers = FunctionSampler(func=utils.outlier_rejection)#para cambiar la seed, en el utils

discretizer = KBinsDiscretizer(n_bins=3, strategy="uniform")

## 2.2 Selección de modelos

Para esta selección, es muy importante escoger los mejores hiperparámteros para cada modelo, por ello utilizaremos un grid de hiperparámetros que evaluará diferentes combinaciones de los hiperparámetros y las ejecutará para luego poder ver el resultado de cada combinación, todo esto junto con un pipeline para el procesamiento de los datos.

### 2.2.1 Vecinos más cercanos

In [ ]:
#Modelo
estimator = clone(k_neighbors_model)

#Hiperparámetros
weights = ["uniform", "distance"]
n_neighbors = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
metric = ["euclidean","manhattan"]

Sobre `weights`, poco que decir, esos son sus dos únicos valores, cuya principal diferencia es la ponderación o no dependiendo de la distancia, resulta un hiperparámetro bastante útil.

Sobre `n_neighbors`, lo primero, que hemos descartado los valores 1 y 2 debido a sobreajustes. Además, como los datos no son muy grandes (el train lo forman 537 datos), hemos establecido un array secuencial, ya que knn recomienda evaluar de forma secuencial desde 1 hasta máximo la raíz de N (23).

Sobre `metric` queríamos establecer al menos 2 métricas no muy complejas para el estudio de distancias, luego de la lista que proporciona scikit, escogimos estas 2.

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, estimator)

k_neighbors_clf = utils.optimize_params(pipeline,
                                        X, y, cv,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors,
                                        kneighborsclassifier__metric=metric)

Según nuestro método de validación esta combinación acaba siendo la mejor, incluso si consideramos que al ser una enfermedad nos interesa más conocer el caso de positivos marcados como tal respecto al total de positivos (Recall), esta combinación se encuentra en el puesto 8, luego no es tampoco un mal resultado.

También comentar que este conjunto de datos es pequeño, los datos se encuentran más o menos dispersos y sin discretizar, lo que provoca que tanto `uniform` como `distance`, estén más o menos reñidos. Además de que `metric=manhattan` acaba siendo la mejor métrica con diferencia.

También comenta que aunque conseguimos buenos puestos validando con `Accuracy` y `REcall`, en `AUC` por desgracia no ocurre lo mismo:

* Puesto `Accuracy` = 2
* Puesto `AUC` = 57
* Puesto `Recall` = 8

**Discretizado**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, discretizer, estimator)

k_neighbors_clf_ds = utils.optimize_params(pipeline,
                                        X, y, cv,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors,
                                        kneighborsclassifier__metric=metric)

Discretizando, acabamos obteniendo peor `score` que sin discretizar, podemos observar que en este caso el `n_neighbors` es menor y además provoca que las 2 variables de `metric` estén más reñidas, tiene sentido ya que al discretizar dividimos en intervalos, luego esto afecta bastante a los 3 hiperparámetros utilizados, ya que todos están relacionados con las distancias.

Validando obtenemos un caso parecido al anterior pero con algo que nos puede interesar, obtenemos peor `Accuracy` y mejor `Recall`, en `AUC` por desgracia sigue igual, esto es interesante porque como la base de datos trata una enfermedad, `Recall` podría interesarnos más al tener mejor puesto, aún así su score sigue siendo menor al caso sin discretizar:

* Puesto `Accuracy` = 15
* Puesto `AUC` = 79
* Puesto `Recall` = 2

### 2.2.2 Árboles de decisión

**Árbol totalmente expandido**

In [ ]:
utils.plot_tree(decision_tree_model, X, y)

In [ ]:
#Modelo
estimator = clone(decision_tree_model)

#Hiperparámetros
criterion = ["gini", "entropy"]
max_depth = [2, 3, 4, 5, 6, 7, 8, 9]
ccp_alpha = [0.0, 0.015, 0.025, 0.035]
splitter = ["best", "random"]

En relación con el árbol expandido totalmente podemos observar que la profundidad máxima es 12, de aquí establecemos un corte más o menos moderado en `max_depth` hasta un máximo de 9 para evitar sobreajustes, además de que omitimos profundidad 0 y 1 ya que es a partir de la profundidad 2 donde de verdad podemos empezar a considerar las particiones, podemos observar un clara división de variables a izquierda y derecha (en el subárbol izquierdo predomina el naranja mientras que en el derecho el azul)

Además hemos cambiado los valores de `ccp_alpha`, los hemos establecido más pequeños debido a que en guías como las de scikit podemos observar que con valores muy bajos ya comienza a reducir el número de nodos, además de que también hemos observado de que conseguimos mejores scores que con los valores anteriormente por defecto.

Además también hemos considerado `criterion` y `splitter`, ambos relacionados con las divisiones (`criterion ` mide la calidad y `splitter` escoge la mejor división dependiendo del valor seleccionado).


**Sin discretizar**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, estimator)

decision_tree_clf = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha,
                                          decisiontreeclassifier__splitter=splitter)

Podemos observar que con este clasificador mejoramos el mejor caso de score anterior, respecto a las salidas, podemos ver que sin discretizar `criterion=entropy` domina los primeros puestos.

Los mejores casos son a partir de `max_depth` mayores o iguales a 4, si nos fijamos en el árbol totalmente expandido podemos observar lo anteriormente comentado sobre la división de colores, donde en la izquierda predomina el naranja y en la derecha el azul, que hacen referencia a los 0's y 1's de nuestra variable clase, luego es normal que obtengamos mejores resultados a partir de esas profundidades.

Con este clasificador mejoramos el mejor caso de score anterior, esto se puede ver reflejado en los puestos, con este clasificador conseguimos más filas debido a las configuraciones pero aún así acabamos obteniendo mejores puestos:

* Puesto `Accuracy` = 2
* Puesto `AUC` = 3
* Puesto `Recall` = 8

**Discretizado**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, discretizer, estimator)

decision_tree_clf_ds = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha,
                                          decisiontreeclassifier__splitter=splitter)

Aquí vuelve a pasar lo mismo que en el antiguo modelo, discretizando perdemos, pero aún así superamos `k_neighbors_model` discretizado.

La discretización acaba provocando que esta vez el `criteron` predominante sea gini, debido a la división que esta provoca. Antes también encontrábamos un `ccp_alpha` variado, mientras que aquí predominan los casos donde no se poda, además la división también provoca que los mejores árboles sean los de profundidad pequeña, de ahí que consigamos un peor score.

En este caso conseguimos muy buenos scores de `Accuracy` y `AUC` pero uno bastante malo de `Recall` que para este tipo de bases de datos resulta muy importante un buen score en este evaluador, sus puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 3
* Puesto `Recall` = 25

### 2.2.3 Adaptative Boosting (AdaBoost)

In [ ]:
#Modelo
estimator = clone(adaboost_model)
base_estimator = clone(decision_tree_model)

#Hiperparámetros
base_estimator = [base_estimator]
learning_rate = [0.85, 0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3, 4, 5]
ccp_alpha = [0.0, 0.015, 0.025, 0.035]

En relación con el árbol totalmente expandido, y sabiendo que `adaboost_model` trabaja con árboles pequeños, hemos establecido un valor máximo para `max_depth` de 5.

Para los árboles, seguimos teniendo en cuenta los 2 `criterion` anteriores y los valores de `ccp_alpha`.

Para el `learning_rate`, donde se marca el valor de contribución de un modelo nuevo al ya existente, hemos utilizado valores altos ya que consideramos que deben tener un peso cercano a 1 en la contribución y poder analizar como varía según sea más alto o bajo.

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, estimator)

adaboost_clf = utils.optimize_params(pipeline,
                                     X, y, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

Sin discretizar, podemos observar primeramente que el mejor `criterion` es `gini` y que al ser `adaboost_model`, la mejor `max_depht` acaba siendo 1. Podemos notar algo de similitud con el decision tree discretizado ya que su `max_depht` también es un valor pequeño y el `criterion` acaba siendo gini.

También notamos que en los 3 métodos de validación utilizados, estamos consiguiendo cada vez resultados mejores, de ahí el aumeto de score respecto a los casos anteriores, los puestos:

* Puesto `Accuracy` = 15
* Puesto `AUC` = 8
* Puesto `Recall` = 3

**Discretizado**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, discretizer, estimator)

adaboost_clf_ds = utils.optimize_params(pipeline,
                                     X, y, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

Seguimos obteniendo peores resultados discretizando, la combinación ganadora sigue manteniendo un `criterion=gini` y `max_depth=1`, en este caso no poda y el `learning_rate=1`.

También encontramos más combinaciones de los 2 tipos de criterio en los mejores casos y el significativo rechazo a la poda en los mejores casos, tiene sentido debido al discretizador, por la división de los datos.

* Puesto `Accuracy` = 27
* Puesto `AUC` = 23
* Puesto `Recall` = 1

### 2.2.4 Bootstrap Aggregating (Bagging)

In [ ]:
#Modelo
estimator = clone(bagging_model)
base_estimator = clone(decision_tree_model)

#Hiperparámetros
base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
n_estimators = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
bootstrap_features = [True, False] 
bootstrap = [True, False] 

Debido a que `bagging_model`, trabaja mejor con árboles profundos, favorecemos las creaciones totales de los árboles, dejando los parámetros por defecto, con la excepción de `criterion`.

Debido a que, teóricamente, cuantos más estimadores, mejor score obtenemos, `n_estimators` es el hiperparámetro que más nos interesa, de ahí las combinaciones, a parte, hemos tenido en cuenta `boostrap_feautres` y `boostrap`, ambas tratan las variables con reemplazo, para evaluar los posibles resultados.

También se tuvieron en cuentra otros hiperparámetros pero se descartaron debido a que no podrucían ningún tipo de cambio, como `warm_start` que si se establece como `warm_start=True` reutiliza la solución de la llamada anterior para ajustar y agregar más estimadores al conjunto, otro que también se estimó pero no podrucía cambios era `oob_score` que marcado como `oob_score=True` tendría en cuenta las muestras fuera de la bolsa para estimar el error de generalización (Detallar que otros modelos tratados en esta práctica también poseen estos hiperparámetros pero el resultado seguía siendo el mismo, no producían cambios a pesar de que son muy interesantes).

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, estimator)

bagging_clf = utils.optimize_params(pipeline,
                                    X, y, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__base_estimator__criterion=criterion,
                                    baggingclassifier__n_estimators=n_estimators,
                                    baggingclassifier__bootstrap_features =bootstrap_features,
                                    baggingclassifier__bootstrap=bootstrap)

Continuando con la racha de que cada algoritmo obtiene un mejor score que el anterior, `bagging_model` sin discretzar obtiene mejor tasa que `adaboost_model`.

En este caso observamos significantes parecidos en los principales resultados, los cuáles son `criterion=entropy`, `bootstrap=True` y `boostrap_features=False` (estos 2 últimos son los valores por defecto).

Además los mejores valores de `n_estimators`, el cuál por defecto es igual a 10, acaban siendo mucho mayores, aunque también podemos observar que el ganador acaba teniendo `n_estimators=80`, lo que deja por debajo los valores de 90 y 100, lo que nos podría hacer llegar a pensar de que hemos alcanzado el tope.

Sus puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 2
* Puesto `Recall` = 1

**Discretizado**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, discretizer, estimator)

bagging_clf_ds = utils.optimize_params(pipeline,
                                    X, y, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__base_estimator__criterion=criterion,
                                    baggingclassifier__n_estimators=n_estimators,
                                    baggingclassifier__bootstrap_features =bootstrap_features,
                                    baggingclassifier__bootstrap=bootstrap)

Seguimos obteniendo peores resultados en discretización, además la dicretización anterior, la de `adaboost_model` es mejor que esta.

Acabamos obteniendo la misma solución que sin discretizar, excepto que el `n_estimators=90` discretizando.

En este caso las combinaciones de los hiperparámetros de los mejores casos, son muy parecidas a las de los resultados sin discretizar. 

Teniendo en cuenta estos resultados, podríamos omitir el uso de los hiperparámetros `bootstrap` y `bootstrap_features`, ya que no tienen peso significativo en los mejores resultados, aún así hemos deicido dejarlas a modo de datos interesantes ya que estos de verdad proporcionan variaciones en los resultados.

Los puestos en este caso son:

* Puesto `Accuracy` = 11
* Puesto `AUC` = 21
* Puesto `Recall` = 4

### 2.2.5 Random Forests

In [ ]:
#Modelo
estimator = clone(random_forest_model)

#Hiperparámetros
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
n_estimators = [100, 150, 200, 250]

Random Forests es otro de los modelos que trabaja mejor con árboles profundos, luego dejaremos los parámetros por cefecto y nos centraremos en los proporcionados por la práctica, además de aumentar el número de `n_estimators`.

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, estimator)

random_forest_clf = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__max_features=max_features,
                                          randomforestclassifier__n_estimators=n_estimators)

En este caso, aunque sea por un 0.001, `random_forest_model` sin discretizar acaba obteniendo peor score que el mejor de lo que llevamos de modelos.

Entre los mejores casos, podemos ver que en `criterion` sus dos valores están bastante reñidos, en `max_features` acaba predominando el valor log2 y la mayoría de los mejores resultados, acaban siendo mayores al valor por defecto `n_estimators=100`, a pesar de esto una de las combinaciones con `n_estimators=100` acaba en segunda posición.

Destacar también que en este modelo sin discretizar se han obtenido muy buenos resultados en general en el train de la validación `AUC`.

Los puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 11
* Puesto `Recall` = 1

**Discretizado**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, discretizer, estimator)

random_forest_clf_ds = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__max_features=max_features,
                                          randomforestclassifier__n_estimators=n_estimators)

Discretizar sigue proporcionando peores scores, en este caso la solución es totalmente distinta a la anterior.

Además podemos ver algo realmente interesante, una de los hiperparámetros no proporciona ningún tipo de variación debido a la discretización, este hiperparámetro es `max_features`, la cuál aquí podríamos suprimirla ya que no aporta nada.

Además en este caso los `n_estimators` mayores tienen más peso que sin discretizar, lo que provoca que, aunque gini predomine en los mejores casos, entropy acaba siendo el mejor.

Además esta discretización pierde contra la anterior de `bagging_model` discretizado.

Los puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 1
* Puesto `Recall` = 3

### 2.2.6 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
#Modelo
estimator = clone(gradient_boosting_model)

#Hiperparámetros
learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3, 4]
ccp_alpha = [0.0, 0.5, 1]
n_estimators = [100, 150]

`gradient_boosting_model` trabaja mejor con árboles pequeños, de ahí que limitemos el tamaño a un máximo de 4, además hemos establecido valores intermedios en `learning_rate` y `ccp_alpha` para comprobar las variaciones, ya que para que no se extienda mucho este modelo no hemos modificado mucho los hiperparámetros ya establecidos por la práctica que no eran malos.

Además de que queríamos establecer al menos 2 valores distintos de `n_estimators` ya que puede provocar mejorías en los resultados al aumentarlo.

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, estimator)

gradient_boosting_clf = utils.optimize_params(pipeline,
                                              X, y, cv,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__n_estimators=n_estimators,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha)

Hasta aquí, este acaba siendo el mejore score obtenido de todos los modelos tenidos en cuenta.

Podemos observar también que los mejores parámetros no utilizan poda, aún así, no observamos sobreajuesta en ninguna de las 3 variaciones.

Los mejores valores no pasan de`max_depth=3` debido a lo ya comentado y el `learning_rate` es bastante variado.

También podemos observar que el hiperparámetro proporcionado de la práctica `criterion` en los mejores valores es irrelevante, ya que no produce cambios.

A pesar de estos puestos las diferencias con los primeros son bastante despreciables:

* Puesto `Accuracy` = 5
* Puesto `AUC` = 5
* Puesto `Recall` = 3

**Discretizado**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, discretizer, estimator)

gradient_boosting_clf_ds = utils.optimize_params(pipeline,
                                              X, y, cv,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__n_estimators=n_estimators,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha)

Discretizano seguimos obtenienod peores scores, en este caso los mejores scores no realizan poda.


También podemos observar que en el hiperparámetro proporcionado de la práctica `criterion` ocurre lo mismo que sin discretizar en los mejores valores es irrelevante, ya que no produce cambios.

Los puestos:

* Puesto `Accuracy` = 21
* Puesto `AUC` = 29
* Puesto `Recall` = 8

### 2.2.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

No es posible hacer una discretización de este modelo

In [ ]:
#Modelo
estimator = clone(hist_gradient_boosting_model)

#Hiperparámetros
learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
max_iter = [100, 150, 200]

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(imputar, eliminar_columnas, eliminar_outliers, estimator)

hist_gradient_boosting_clf = utils.optimize_params(pipeline,
                                                   X, y, cv,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__max_iter=max_iter,
                                                   histgradientboostingclassifier__max_leaf_nodes=max_leaf_nodes)

Recalcar que aquí acaba empeorando bastante el score respecto a diferentes modelos que no disctrizan, los mejores casos los encontramos con los valores más bajos de `max_iter` mientras que en el resto de hiperparámetros encontramos más variedad en los mejores casos.

Los puestos:

* Puesto `Accuracy` = 13
* Puesto `AUC` = 29
* Puesto `Recall` = 1

## 2.3 Construcción y validación del modelo final

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf,
    "Nearest neighbors discretizado": k_neighbors_clf_ds,
    "Decision tree discretizado": decision_tree_clf_ds,
    "AdaBoost discretizado": adaboost_clf_ds,
    "Bagging discretizado": bagging_clf_ds,
    "Random Forests discretizado": random_forest_clf_ds,
    "Gradient Boosting discretizado": gradient_boosting_clf_ds
}

In [ ]:
utils.evaluate_estimators(estimators, X, y)

Lo primero a destacar es que discretizano obtenemos peores resultados en `Diabetes`, eso si, algunas discretizaciones son mejores que otros modelos sin discretizar y podrían tenerse en cuenta.

Además en los resultados marcábamos a `gradient_boosting_model` como el mejor, mientras que después de evaluar, acaba siendo `bagging_model` seguido muy de cerca de `random_forests_model`, como el mejor clasificador con un total de un 85%.

Otro factor a tener cuenta, que hemos descartado tanto aquí como en `Wisconsin` es el tiempo, ya que estos conjuntos de datos son muy pequeños, luego al final acabamos despreciando los tiempos, cosa que en conjuntos de datos mucho mayores debe tenerse muy en cuenta.

# 3. Wisconsin

## 3.1 Carga de datos

Cargamos datos y comprobamos que se han cargado correctamente

In [ ]:
filepath= "../input/breast-cancer-wisconsin-data/data.csv"
target = "diagnosis"

data = utils.load_data(filepath,None,target)
data = data.drop(columns=['Unnamed: 32'])
pd.get_dummies(data, "diagnosis")
data = pd.get_dummies(data, columns = ["diagnosis"], drop_first = True)
data.rename(columns={'diagnosis_M': 'diagnosis'}, inplace=True)
data.index=range(data.shape[0])

data.sample(8,random_state=random_state)

A continuación, obtendremos nuestras variables predictoras por un lado y la variable objetivo por otro.

In [ ]:
target = "diagnosis"

(X, y) = utils.divide_dataset(data, target)

Ahora comprobamos que se han separado correctamente:

**Variables predictoras**

In [ ]:
X.sample(5, random_state=random_state)

**Variable clase**

In [ ]:
y.sample(5, random_state=random_state)

Ahora, divididiremos el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado (70% train y 30% test):

In [ ]:
stratify = y
train_size = 0.7

(X_train, X_test, y_train, y_test) = train_test_split(X, y,
                                                      stratify=stratify,
                                                      train_size=train_size,
                                                      random_state=random_state)

Ahora comprobamos que hemos separado correctamente el conjunto en train y test:

**Train - Variables Predictoras**

In [ ]:
X_train.sample(5, random_state=random_state)

**Train - Variable Clase**

In [ ]:
y_train.sample(5, random_state=random_state)

**Test - Variables Predictoras**

In [ ]:
X_test.sample(5, random_state=random_state)

**Tes - Variable Clase**

In [ ]:
y_test.sample(5, random_state=random_state)

In [ ]:
#Establecemos la 'X' y la 'y' de esta forma por comodidad para los modelos

X = X_train
y = y_train

**Parámetros del pipeline**

In [ ]:
eliminar_outliers = FunctionSampler(func=utils.outlier_rejection)#para cambiar la seed, en el utils

discretizer = KBinsDiscretizer(n_bins=3, strategy="quantile")

## 3.2 Selección de modelos

Para este conjunto de datos utilizaresmo los mismos hiperparámetros que en el conjunto de datos anterior para compararlos, a excepción de `n_neighbors` de `k_neighbors_model` debido a que hemos seguido un array secuencial que tiene como valor máximo la raíz del número de datos a tratar y de `max_depth` en los modelos que lo implementen debido a que el árbol es distinto (por razones obvias).

### 3.2.1 Vecinos más cercanos

In [ ]:
#Modelo
estimator = clone(k_neighbors_model)

#Hiperparámetros
weights = ["uniform", "distance"]
n_neighbors = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
metric = ["euclidean","manhattan"]

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, estimator)

k_neighbors_clf = utils.optimize_params(pipeline,
                                        X, y, cv,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors,
                                        kneighborsclassifier__metric=metric)

Podemos observar que sin discretizar, conseguimos un score similar al de diabetes en este modelo, salvo que en este caso `n_neighbors` ha sido menor y el `weights` escogido ha sido `distance`.

En esta conjunto de datos, pasa lo mismo que el anterior en `metric`, `manhattan` sigue predominando y la mayor diferencia que podemos destacar es que `wights=distance` tiene más peso en los mejores casos en este conjunto de datos que en el anterior.

Los puestos obtendios:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 9
* Puesto `Recall` = 1

**Discretizado**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, discretizer, estimator)

k_neighbors_clf_ds = utils.optimize_params(pipeline,
                                        X, y, cv,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors,
                                        kneighborsclassifier__metric=metric)

Lo más llamativo de este caso es el score, conseguimos un gran score de un 95% de acierto, en comparación con el otro conjunto de datos, donde discretizando obteníamos muy malos scores, esto puede deberse principalmente a que solo imputamos outliers en este conjunto de datos, además de que también poseemos 32 columnas y otro tipo de discretización. Además de que provoca un empate en el primer puesto, dado que ambas métricas consiguen el mismo resultado.

Sobre el mejor resultado, acaba siendo el mismo que en el caso del conjunto de datos anterior discretizando, solo que el `n_neighbors` en este caso es mayor, además las `metric` están más reñidas, y el mejor `weights` por diferencia es `distance` si tenemos en cuenta el empate, ya que ocupa los primeros puestos.

Los puestos obtendios:

* Puesto `Accuracy` = 7
* Puesto `AUC` = 45
* Puesto `Recall` = 1

### 3.2.2 Árboles de decisión

**Árbol totalmente expandido**

In [ ]:
utils.plot_tree(decision_tree_model, X, y)

In [ ]:
#Modelo
estimator = clone(decision_tree_model)

#Hiperparámetros
criterion = ["gini", "entropy"]
max_depth = [2, 3, 4]
ccp_alpha = [0.0, 0.015, 0.025, 0.035]
splitter = ["best", "random"]

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, estimator)

decision_tree_clf = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha,
                                          decisiontreeclassifier__splitter=splitter)

Respecto al modelo anterior sin discretizar, conseguimos un gran salto en el score, también se debe entre otras cosas a la complejidad del árbol, ya que si nos fijamos es un árbol bastante simple, no muy complejo como en el caso anterior.

En el modelo de `Diabetes` sin discretizar también coincide en que el mejor `criterio=entropy` y `splitter=best`, es normal que no coincida el `max_depth` como ya hemos comentado, y aquí utiliza un valor de poda superior.

Los puestos obtendios:

* Puesto `Accuracy` = 2
* Puesto `AUC` = 25
* Puesto `Recall` = 1

**Discretizado**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, discretizer, estimator)

decision_tree_clf_ds = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha,
                                          decisiontreeclassifier__splitter=splitter)

Discretizando también conseguimos un buen score, en este caso pierde contra el modelo sin discretizar pero ambos consiguen un score superior a los 2 obtenidos en diabetes.

Además lo único que cambia en las 2 mejores scores es el `max_depth`.

Aquí el `criterion=entropy`es el más utilizado en los mejores casos, como sucede sin discretizar, `splitter` se comporta en ambos conjuntos de datos igual.

Comparando con la discretización de este modelo en diabetes, aquí poda y el `criterion` es distinto.

Los puestos obtendios:

* Puesto `Accuracy` = 6
* Puesto `AUC` = 5
* Puesto `Recall` = 1

### 3.2.3 Adaptative Boosting (AdaBoost)

In [ ]:
#Modelo
estimator = clone(adaboost_model)
base_estimator = clone(decision_tree_model)

#Hiperparámetros
base_estimator = [base_estimator]
learning_rate = [0.85, 0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3, 4]
ccp_alpha = [0.0, 0.015, 0.025, 0.035]

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, estimator)

adaboost_clf = utils.optimize_params(pipeline,
                                     X, y, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

Adaboost también consigue un muy buen score, comparando este caso con el de `Diabetes`, los mejores scores solo coinciden en el `max_depth`, no es de extrañar ya que este clasificador trabaja con árboles pequeños, y los de profundidad 1 son muy comunes.

Comparando con `Diabetes`, también podemos observar que mientras que en `Diabetes` el `criterion=gini` predominaba en los mejores scores, aquí acaba predominando `criterion=entropy`.

Los puestos obtendios:

* Puesto `Accuracy` = 12
* Puesto `AUC` = 34
* Puesto `Recall` = 1

**Discretizado**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, discretizer, estimator)

adaboost_clf_ds = utils.optimize_params(pipeline,
                                     X, y, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

El score discretizando acaba siendo peor que sin discretizar, aún así nos colocamos en un 94%, las diferencias respecto a no discretizar son que aquí `criterion=gini` (antes los mejores casos eran `entropy` y discretizando produce lo contrario, los mejores casos utilizan `criterion=gini`) y realizamos poda.

La única similitud que podemos sacar del caso discretizado en `Diabetes` es que `max_depth=1` en los mejores casos, cosa que se debe por el clasificador como ya hemos comentado.

Los puestos obtendios:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 43
* Puesto `Recall` = 1

### 3.2.4 Bootstrap Aggregating (Bagging)

In [ ]:
#Modelo
estimator = clone(bagging_model)
base_estimator = clone(decision_tree_model)

#Hiperparámetros
base_estimator = [base_estimator]
criterion = ["gini", "entropy"]
n_estimators = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
bootstrap_features = [True, False] 
bootstrap = [True, False] 

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, estimator)

bagging_clf = utils.optimize_params(pipeline,
                                    X, y, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__base_estimator__criterion=criterion,
                                    baggingclassifier__n_estimators=n_estimators,
                                    baggingclassifier__bootstrap_features =bootstrap_features,
                                    baggingclassifier__bootstrap=bootstrap)

Seguimos con buenos scores, además aquí se nota que cuánto mayor es el `n_estimators`, mejor es el score obtenido, además, comparando con `Diabetes` obtenemos una gran diferencia relativa a `bootstrap` y `bootstrap_features`, ya que la combinación con mejores scores son ambas a `True`, si bien es verdad que en `Wisconsin` podemos notar buenos scores también con `criterion=gini` cosa que en `Diabetes` no ocurre ya que predomina `entropy`

Los puestos obtendios:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 6
* Puesto `Recall` = 1

**Discretizado**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, discretizer, estimator)

bagging_clf_ds = utils.optimize_params(pipeline,
                                    X, y, cv,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__base_estimator__criterion=criterion,
                                    baggingclassifier__n_estimators=n_estimators,
                                    baggingclassifier__bootstrap_features =bootstrap_features,
                                    baggingclassifier__bootstrap=bootstrap)

Discretizando seguimos con buenos scores, se sigue manteniendo que cuánto mayor es el `n_estimators` mejor es el score, aunque en este caso haya sido 90 y sin discretizar 100. Las diferencias entre los mejores casos discretizando y sin discretizar lo marcan realmente `criterion=entropy` (ambos casos ganadores lo poseen) quien predomina en este caso y `bootstrap_features` que en este caso podemos encontrar valores False en los mejores casos.

El mejor resultado acaba coincidiendo exacatamente con el obtenido en `Diabetes` discretizando a pesar de que encontramos diferencias en los 2 hiperparámetros comentados en la comparación con `Wisconsin` sin discretizar.

Los puestos obtendios:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 34
* Puesto `Recall` = 1

### 3.2.5 Random Forests

In [ ]:
#Modelo
estimator = clone(random_forest_model)

#Hiperparámetros
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
n_estimators = [100, 150, 200, 250]

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, estimator)

random_forest_clf = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__max_features=max_features,
                                          randomforestclassifier__n_estimators=n_estimators)

Seguimos manteniendo buen score, los hiperparámetros se encuentran bastante reñidos en general en los mejores casos, aunque aquí tenemos el caso peculiar de que aunque aumentemos el `n_estimators`, en este caso el mejor acaba siendo el valor por defecto que quedó en segunda posicioon en `Diabetes` sin discretizar `n_estimators=100`.


Los puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 9
* Puesto `Recall` = 4

**Discretizado**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, discretizer, estimator)

random_forest_clf_ds = utils.optimize_params(pipeline,
                                          X, y, cv,
                                          randomforestclassifier__criterion=criterion,
                                          randomforestclassifier__max_features=max_features,
                                          randomforestclassifier__n_estimators=n_estimators)

Aunque empeora un poco el score, tampoco lo podemos calificar de malo, destacar que respecto al caso sin discretizar, los hiperparámetros de los mejores casos, se comportan igual, salvo `n_estimators` ya que aquí si que tienen mayor peso los valores más grandes. 

Este comportamiento es similar al obtenido en los mejores resultados de `Diabetes` discretizando, con la salvedad de que en `Diabetes` el segundo puesto tiene `n_estimators=100`

Los puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 3
* Puesto `Recall` = 2

### 3.2.6 Gradient Tree Boosting (Gradient Boosting)

In [ ]:
#Modelo
estimator = clone(gradient_boosting_model)

#Hiperparámetros
learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3, 4]
ccp_alpha = [0.0, 0.5, 1]
n_estimators = [100, 150]

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, estimator)

gradient_boosting_clf = utils.optimize_params(pipeline,
                                              X, y, cv,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__n_estimators=n_estimators,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha)

Seguimos manteniendo buenos scores.

Podemos observar que los mejores parámetros no utilizan poda, no pasan de`max_depth=3` al igual que en `Diabetes`.

El `learning_rate=0.1` es el más utilizado en los mejores casos.

También podemos observar que ne `Wisconsin`el hiperparámetro proporcionado de la práctica `criterion` en los mejores valores no es irrelevante, ya que no produce cambios aunque sean muy pequeños.

Los puestos

* Puesto `Accuracy` = 1
* Puesto `AUC` = 1
* Puesto `Recall` = 1

**Discretizado**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, discretizer, estimator)

gradient_boosting_clf_ds = utils.optimize_params(pipeline,
                                              X, y, cv,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__n_estimators=n_estimators,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha)

Discretizando seguimos obteniendo buenos scores, aquí los mejores casos siguen mantiendo el criterio de no podar.


El resto de hiperparámetros se comportan de la misma forma que sin discretizar, salvo que en `max_depth` si seguimos mateniendo que los mejores casos utilizan profundidades de 1 a 3, estos incluidos, lo cumplen de manera descendente, es decir de 3 a 1.

Los puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 17
* Puesto `Recall` = 1

### 3.2.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)

In [ ]:
#Modelo
estimator = clone(hist_gradient_boosting_model)

#Hiperparámetros
learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]
max_leaf_nodes = [15, 31, 65, 127]
max_iter = [100, 150, 200]

**Sin discretizar**

In [ ]:
pipeline = make_pipeline(eliminar_outliers, estimator)

hist_gradient_boosting_clf = utils.optimize_params(pipeline,
                                                   X, y, cv,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__max_iter=max_iter,
                                                   histgradientboostingclassifier__max_leaf_nodes=max_leaf_nodes)

Seguimos obteniendo buenos scores, los mejores casos los encontramos con los valores más altos de `max_iter` además de que el mejor `learning_rate=0.05` mientras que en el resto de hiperparámetros encontramos más variedad en los mejores casos.

Los puestos:

* Puesto `Accuracy` = 1
* Puesto `AUC` = 9
* Puesto `Recall` = 1

## 3.3 Construcción y validación del modelo final

In [ ]:
estimators = {
    "Nearest neighbors": k_neighbors_clf,
    "Decision tree": decision_tree_clf,
    "AdaBoost": adaboost_clf,
    "Bagging": bagging_clf,
    "Random Forests": random_forest_clf,
    "Gradient Boosting": gradient_boosting_clf,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf,
    "Nearest neighbors discretizado": k_neighbors_clf_ds,
    "Decision tree discretizado": decision_tree_clf_ds,
    "AdaBoost discretizado": adaboost_clf_ds,
    "Bagging discretizado": bagging_clf_ds,
    "Random Forests discretizado": random_forest_clf_ds,
    "Gradient Boosting discretizado": gradient_boosting_clf_ds
}

In [ ]:
utils.evaluate_estimators(estimators, X, y)

Lo primero a destacar es que discretizano obtenemos mucho mejores resultados que en `Diabetes`, también destacar que disminuimos el n_bins debido a que sobreajustaba más de la cuenta y podemos ver que en algunos de `Wisconsin`, destacando el metodo de validación de `AUC` se sobreajusta bastante.

El mejor clasificador acaba siendo un triple empate entre `adabbost_model`, `bagging_model` y `Random_forests_model`. Luego por tiempos, aunque hayamos dicho que son despreciables en este conjunto de datos, nos quedaríamos con `adabbost_model`.

# 4. Estudio de un kernel de Kaggle

Este kernel de Kaggle, está dedicado a personas que estás comenzando a utilizar la ciencia de datos, básicamente es un tutorial muy completo para que podamos conseguir bastante experiencia en este ámbito. (no ha dado tiempo a comprobar su ejecución)

[A Data Science Framework: To Achieve 99% Accuracy](https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy/notebook)

**Tabla de contenidos**

* Capítulo 1 - Definir el problema
* Capítulo 2 - Preparar los datos para el consumo
* Capítulo 3 - Limpieza de datos: corregir, completar, crear y convertir
* Capítulo 4 - Datos del modelo
* Capítulo 5 - Evaluar el desempeño del modelo
* Capítulo 6 - Ajustar modelo con hiperparámetros
* Capítulo 7 - Ajuste del modelo con selección de funciones
* Capítulo 8 - Validar e implementar

## 4.1 Definir el problema 

Nos encontramos ante un probema basado en el incidente de Titanic, debemos desarrollar un algoritmo para predecir los supervivientes de este navío. [DATASET](https://www.kaggle.com/c/titanic/data)

## 4.2 Preparar los datos para el consumo 

In [ ]:

#COMIENZA REALIZANDO LA CARGA DE LIBERÍAS PARA EL PREPROCESAMIENTO

import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

#ADEMÁS YA OBTIENE EL TRAIN Y EL TEST DIVIDIDO COMO PODEMOS COMPROBAR EN ELOS ARCHIBOS DE TITANIC
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#AQUÍ PODEMOS VER QUE CARGA LIBRERÍAS SOBRE MODELOS Y 
#GRÁFICAS PARA EL ANÁLISIS ANTES DE EJECUTAR LOS MODELOS

#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

## 4.3 Limpieza de datos: corregir, completar, crear y convertir

Aquí realiza la carga de los datos y podemos observar que tiene un total de 891 entradas, además de que muchas variables predictoras tienen valores núlos, la variable objetivo es `Survived` y puede tomar el valor 0 o 1, luego estamos ante un problema categórico.

In [ ]:
#import data from file: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
data_raw = pd.read_csv('../input/titanic/train.csv')


#a dataset should be broken into 3 splits: train, test, and (final) validation
#the test file provided is the validation file for competition submission
#we will split the train set into train and test data in future sections
data_val  = pd.read_csv('../input/titanic/test.csv')


#to play with our data we'll create a copy
#remember python assignment or equal passes by reference vs values, so we use the copy function: https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs
data1 = data_raw.copy(deep = True)

#however passing by reference is convenient, because we can clean both datasets at once
data_cleaner = [data1, data_val]


#preview data
print (data_raw.info()) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html
#data_raw.head() #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html
#data_raw.tail() #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.tail.html
data_raw.sample(10) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html

Aquí observaresmos lo aneterior comentado, observa qué variables tienen valores nulos y cuántos son, tanto en test como en train.

Las columnas son `Age`, `Cabin` y `Embarked` en train y `Age`, `Cabin` y `Fare` en test.

In [ ]:
print('Train columns with null values:\n', data1.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum())
print("-"*10)

data_raw.describe(include = 'all')

Ahora pasa a la limpieza de los datos, obtando por rellenar los nulos de `Age`, `Fare` por la media de los valores de esas columnas.

Las columnas `PassengerID` y `Ticket` son eliminadas debido a que son ids que no aportan nada.

`Sex` y `Embarked` son realmente variables binarias, por lo que las establecerá por 1's y 0's

In [ ]:
###COMPLETING: complete or delete missing values in train and test/validation dataset
for dataset in data_cleaner:    
    #complete missing age with median
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)

    #complete embarked with mode
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

    #complete missing fare with median
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
#delete the cabin feature/column and others previously stated to exclude in train dataset
drop_column = ['PassengerId','Cabin', 'Ticket']
data1.drop(drop_column, axis=1, inplace = True)

print(data1.isnull().sum())
print("-"*10)
print(data_val.isnull().sum())

Los datos nos proporcionan columnas relacionas con nombres (pudiendo derivar de aquí géneros del título, tamaños de la familia según el apelido y el SES de títulos como médico o maestro), los cuales poseen apellidos, número de padres, número de hijos, número de hermanos, número de cónyuges.

Ahora utilizará estos datos para crear nuevas columnas que podrán generar información que le interese.

In [ ]:
###CREATE: Feature Engineering for train and test/validation dataset
for dataset in data_cleaner:    
    #Discrete variables
    dataset['FamilySize'] = dataset ['SibSp'] + dataset['Parch'] + 1

    dataset['IsAlone'] = 1 #initialize to yes/1 is alone
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]


    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)

    #Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)


    
#cleanup rare title names
#print(data1['Title'].value_counts())
stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
title_names = (data1['Title'].value_counts() < stat_min) #this will create a true false series with title name as index

#apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
print(data1['Title'].value_counts())
print("-"*10)


#preview data again
data1.info()
data_val.info()
data1.sample(10)

Ahora realizará los cambios necesarios para realizar esta ingeniería de funciones, como transformar los datos categóricos en `dummy variables` para los análisis matemáticos

In [ ]:
#CONVERT: convert objects to category using Label Encoder for train and test/validation dataset

#code categorical data
label = LabelEncoder()
for dataset in data_cleaner:    
    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])
    dataset['Embarked_Code'] = label.fit_transform(dataset['Embarked'])
    dataset['Title_Code'] = label.fit_transform(dataset['Title'])
    dataset['AgeBin_Code'] = label.fit_transform(dataset['AgeBin'])
    dataset['FareBin_Code'] = label.fit_transform(dataset['FareBin'])


#define y variable aka target/outcome
Target = ['Survived']

#define x variables for original features aka feature selection
data1_x = ['Sex','Pclass', 'Embarked', 'Title','SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'IsAlone'] #pretty name/values for charts
data1_x_calc = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code','SibSp', 'Parch', 'Age', 'Fare'] #coded for algorithm calculation
data1_xy =  Target + data1_x
print('Original X Y: ', data1_xy, '\n')


#define x variables for original w/bin features to remove continuous variables
data1_x_bin = ['Sex_Code','Pclass', 'Embarked_Code', 'Title_Code', 'FamilySize', 'AgeBin_Code', 'FareBin_Code']
data1_xy_bin = Target + data1_x_bin
print('Bin X Y: ', data1_xy_bin, '\n')


#define x and y variables for dummy features original
data1_dummy = pd.get_dummies(data1[data1_x])
data1_x_dummy = data1_dummy.columns.tolist()
data1_xy_dummy = Target + data1_x_dummy
print('Dummy X Y: ', data1_xy_dummy, '\n')



data1_dummy.head()

Ahora que ha limpiado los datos, vuelve a comprobarlos

In [ ]:
print('Train columns with null values: \n', data1.isnull().sum())
print("-"*10)
print (data1.info())
print("-"*10)

print('Test/Validation columns with null values: \n', data_val.isnull().sum())
print("-"*10)
print (data_val.info())
print("-"*10)

data_raw.describe(include = 'all')

In [ ]:
#split train and test data with function defaults
#random_state -> seed or control random number generator: https://www.quora.com/What-is-seed-in-random-number-generation
train1_x, test1_x, train1_y, test1_y = model_selection.train_test_split(data1[data1_x_calc], data1[Target], random_state = 0)
train1_x_bin, test1_x_bin, train1_y_bin, test1_y_bin = model_selection.train_test_split(data1[data1_x_bin], data1[Target] , random_state = 0)
train1_x_dummy, test1_x_dummy, train1_y_dummy, test1_y_dummy = model_selection.train_test_split(data1_dummy[data1_x_dummy], data1[Target], random_state = 0)


print("Data1 Shape: {}".format(data1.shape))
print("Train1 Shape: {}".format(train1_x.shape))
print("Test1 Shape: {}".format(test1_x.shape))

train1_x_bin.head()

Ahora realizaría un anális exploratorio el cuál no vamos a tratar porque es bastante extenso y nos interesa más lo siguiente, aún así debemos tener en cuenta que es un paso muy importante.

Es interesante verlo y estudiarlo detenidamente ya que utiliza diagramas muy variados y pueden resultar algo complejos.

## 4.4 Datos del modelo

Ahora estudia varios modelos y métodos de validación a través de una validación cruzada de 10 repeticiones con 3 carpetas test y 6 carpetas train

In [ ]:
#Machine Learning Algorithm (MLA) Selection and Initialization
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]



#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = data1[Target]

#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, data1[data1_x_bin], data1[Target], cv  = cv_split)

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(data1[data1_x_bin], data1[Target])
    MLA_predict[MLA_name] = alg.predict(data1[data1_x_bin])
    
    row_index+=1

    
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare
#MLA_predict

In [ ]:
#barplot using https://seaborn.pydata.org/generated/seaborn.barplot.html
sns.barplot(x='MLA Test Accuracy Mean', y = 'MLA Name', data = MLA_compare, color = 'm')

#prettify using pyplot: https://matplotlib.org/api/pyplot_api.html
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')
plt.ylabel('Algorithm')

## 4.5 Evaluar el desempeño del modelo

En base a unas cuestiones planteadas por el autor de la libreta, empieza un estudio de los datos que está tratando para ver qué hiperpárametos y modelos pueden ajustarse mejor al problema

**Pregunta 1:** ¿Estabas en el Titanic? En caso afirmativo, la mayoría (62%) murió. Tenga en cuenta que la supervivencia de nuestra muestra es diferente a nuestra población del 68%. No obstante, si asumimos que todos murieron, la precisión de nuestra muestra es del 62%.

**Pregunta 2:** ¿Eres hombre o mujer? Hombre, la mayoría (81%) murió. Mujeres, la mayoría (74%) sobrevivió. Dándonos una precisión del 79%.

**Pregunta 3A** (bajando por la rama femenina con recuento = 314): ¿Estás en la clase 1, 2 o 3? Clase 1, la mayoría (97%) sobrevivió y Clase 2, la mayoría (92%) sobrevivió. Dado que el subgrupo muerto es menor que 10, dejaremos de bajar por esta rama. Clase 3, está incluso en una división 50-50. No se obtiene nueva información para mejorar nuestro modelo.

**Pregunta 4A** (bajando por la rama femenina clase 3 con recuento = 144): ¿Embarcó desde el puerto C, Q o S? Obtenemos un poco de información. C y Q, la mayoría sobrevivió, así que no hubo cambios. Además, el subgrupo muerto es menor que 10, así que pararemos. S, la mayoría (63%) murió. Entonces, cambiaremos a las hembras, clase 3, embarcadas S de asumir que sobrevivieron a asumir que murieron. La precisión de nuestro modelo aumenta al 81%.

**Pregunta 5A** (bajando por la rama femenina de clase 3 embarcada S con recuento = 88): Hasta ahora, parece que tomamos buenas decisiones. Agregar otro nivel no parece obtener mucha más información. Este subgrupo 55 murió y 33 sobrevivieron, dado que la mayoría murió, necesitamos encontrar una señal para identificar a los 33 o un subgrupo para cambiarlos de muertos a sobrevivientes y mejorar la precisión de nuestro modelo. Podemos jugar con nuestras funciones. Una que encontré fue la tarifa 0-8, la mayoría sobrevivió. Es un tamaño de muestra pequeño 11-9, pero se usa a menudo en estadísticas. Mejoramos ligeramente nuestra precisión, pero no mucho para pasar del 82%. Entonces, pararemos aquí.

**Pregunta 3B** (bajando por la rama masculina con recuento = 577): Volviendo a la pregunta 2, sabemos que la mayoría de los hombres murieron. Entonces, estamos buscando una característica que identifique un subgrupo al que la mayoría sobrevivió. Sorprendentemente, la clase o incluso el embarque no importaba como lo hacía para las mujeres, pero el título sí lo hace y nos lleva al 82%. Adivina y revisando otras características, ninguna parece empujarnos más allá del 82%. Entonces, pararemos aquí por ahora.

A raíz de estas preguntas empieza a establecer un código para plasmarlo e intentar mejorarlo

In [ ]:
#IMPORTANT: This is a handmade model for learning purposes only.
#However, it is possible to create your own predictive model without a fancy algorithm :)

#coin flip model with random 1/survived 0/died

#iterate over dataFrame rows as (index, Series) pairs: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iterrows.html
for index, row in data1.iterrows(): 
    #random number generator: https://docs.python.org/2/library/random.html
    if random.random() > .5:     # Random float x, 0.0 <= x < 1.0    
        data1.set_value(index, 'Random_Predict', 1) #predict survived/1
    else: 
        data1.set_value(index, 'Random_Predict', 0) #predict died/0
    

#score random guess of survival. Use shortcut 1 = Right Guess and 0 = Wrong Guess
#the mean of the column will then equal the accuracy
data1['Random_Score'] = 0 #assume prediction wrong
data1.loc[(data1['Survived'] == data1['Random_Predict']), 'Random_Score'] = 1 #set to 1 for correct prediction
print('Coin Flip Model Accuracy: {:.2f}%'.format(data1['Random_Score'].mean()*100))

#we can also use scikit's accuracy_score function to save us a few lines of code
#http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score
print('Coin Flip Model Accuracy w/SciKit: {:.2f}%'.format(metrics.accuracy_score(data1['Survived'], data1['Random_Predict'])*100))

In [ ]:
#group by or pivot table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html
pivot_female = data1[data1.Sex=='female'].groupby(['Sex','Pclass', 'Embarked','FareBin'])['Survived'].mean()
print('Survival Decision Tree w/Female Node: \n',pivot_female)

pivot_male = data1[data1.Sex=='male'].groupby(['Sex','Title'])['Survived'].mean()
print('\n\nSurvival Decision Tree w/Male Node: \n',pivot_male)

In [ ]:
#handmade data model using brain power (and Microsoft Excel Pivot Tables for quick calculations)
def mytree(df):
    
    #initialize table to store predictions
    Model = pd.DataFrame(data = {'Predict':[]})
    male_title = ['Master'] #survived titles

    for index, row in df.iterrows():

        #Question 1: Were you on the Titanic; majority died
        Model.loc[index, 'Predict'] = 0

        #Question 2: Are you female; majority survived
        if (df.loc[index, 'Sex'] == 'female'):
                  Model.loc[index, 'Predict'] = 1

        #Question 3A Female - Class and Question 4 Embarked gain minimum information

        #Question 5B Female - FareBin; set anything less than .5 in female node decision tree back to 0       
        if ((df.loc[index, 'Sex'] == 'female') & 
            (df.loc[index, 'Pclass'] == 3) & 
            (df.loc[index, 'Embarked'] == 'S')  &
            (df.loc[index, 'Fare'] > 8)

           ):
                  Model.loc[index, 'Predict'] = 0

        #Question 3B Male: Title; set anything greater than .5 to 1 for majority survived
        if ((df.loc[index, 'Sex'] == 'male') &
            (df.loc[index, 'Title'] in male_title)
            ):
            Model.loc[index, 'Predict'] = 1
        
        
    return Model


#model data
Tree_Predict = mytree(data1)
print('Decision Tree Model Accuracy/Precision Score: {:.2f}%\n'.format(metrics.accuracy_score(data1['Survived'], Tree_Predict)*100))


#Accuracy Summary Report with http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report
#Where recall score = (true positives)/(true positive + false negative) w/1 being best:http://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score
#And F1 score = weighted average of precision and recall w/1 being best: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
print(metrics.classification_report(data1['Survived'], Tree_Predict))

In [ ]:
#Plot Accuracy Summary
#Credit: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = metrics.confusion_matrix(data1['Survived'], Tree_Predict)
np.set_printoptions(precision=2)

class_names = ['Dead', 'Survived']
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, 
                      title='Normalized confusion matrix')

Este sería el resutado del modelo mental, llegando a un 82%, ahora intentará mejorarlo utilizando una validación cruzada junto con los modelos e hiperparámetros más adecuados a través de un grid, como el que utilizamos en esta práctica

## 4.6 Ajustar modelo con hiperparámetros

El modelo base, al igual que el nuestro, será un `decision_tree_model`

In [ ]:
#base model
dtree = tree.DecisionTreeClassifier(random_state = 0)
base_results = model_selection.cross_validate(dtree, data1[data1_x_bin], data1[Target], cv  = cv_split)
dtree.fit(data1[data1_x_bin], data1[Target])

print('BEFORE DT Parameters: ', dtree.get_params())
print("BEFORE DT Training w/bin score mean: {:.2f}". format(base_results['train_score'].mean()*100)) 
print("BEFORE DT Test w/bin score mean: {:.2f}". format(base_results['test_score'].mean()*100))
print("BEFORE DT Test w/bin score 3*std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))
#print("BEFORE DT Test w/bin set score min: {:.2f}". format(base_results['test_score'].min()*100))
print('-'*10)


#tune hyper-parameters: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
param_grid = {'criterion': ['gini', 'entropy'],  #scoring methodology; two supported formulas for calculating information gain - default is gini
              #'splitter': ['best', 'random'], #splitting methodology; two supported strategies - default is best
              'max_depth': [2,4,6,8,10,None], #max depth tree can grow; default is none
              #'min_samples_split': [2,5,10,.03,.05], #minimum subset size BEFORE new split (fraction is % of total); default is 2
              #'min_samples_leaf': [1,5,10,.03,.05], #minimum subset size AFTER new split split (fraction is % of total); default is 1
              #'max_features': [None, 'auto'], #max features to consider when performing split; default none or all
              'random_state': [0] #seed or control random number generator: https://www.quora.com/What-is-seed-in-random-number-generation
             }

#print(list(model_selection.ParameterGrid(param_grid)))

#choose best model with grid_search: #http://scikit-learn.org/stable/modules/grid_search.html#grid-search
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
tune_model = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), param_grid=param_grid, scoring = 'roc_auc', cv = cv_split)
tune_model.fit(data1[data1_x_bin], data1[Target])

#print(tune_model.cv_results_.keys())
#print(tune_model.cv_results_['params'])
print('AFTER DT Parameters: ', tune_model.best_params_)
#print(tune_model.cv_results_['mean_train_score'])
print("AFTER DT Training w/bin score mean: {:.2f}". format(tune_model.cv_results_['mean_train_score'][tune_model.best_index_]*100)) 
#print(tune_model.cv_results_['mean_test_score'])
print("AFTER DT Test w/bin score mean: {:.2f}". format(tune_model.cv_results_['mean_test_score'][tune_model.best_index_]*100))
print("AFTER DT Test w/bin score 3*std: +/- {:.2f}". format(tune_model.cv_results_['std_test_score'][tune_model.best_index_]*100*3))
print('-'*10)


#duplicates gridsearchcv
#tune_results = model_selection.cross_validate(tune_model, data1[data1_x_bin], data1[Target], cv  = cv_split)

#print('AFTER DT Parameters: ', tune_model.best_params_)
#print("AFTER DT Training w/bin set score mean: {:.2f}". format(tune_results['train_score'].mean()*100)) 
#print("AFTER DT Test w/bin set score mean: {:.2f}". format(tune_results['test_score'].mean()*100))
#print("AFTER DT Test w/bin set score min: {:.2f}". format(tune_results['test_score'].min()*100))
#print('-'*10)

## 4.7 Ajuste del modelo con selección de funciones

Ahora lo que realiza es una selección de características, eliminando las recursivas.

In [ ]:
#base model
print('BEFORE DT RFE Training Shape Old: ', data1[data1_x_bin].shape) 
print('BEFORE DT RFE Training Columns Old: ', data1[data1_x_bin].columns.values)

print("BEFORE DT RFE Training w/bin score mean: {:.2f}". format(base_results['train_score'].mean()*100)) 
print("BEFORE DT RFE Test w/bin score mean: {:.2f}". format(base_results['test_score'].mean()*100))
print("BEFORE DT RFE Test w/bin score 3*std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))
print('-'*10)



#feature selection
dtree_rfe = feature_selection.RFECV(dtree, step = 1, scoring = 'accuracy', cv = cv_split)
dtree_rfe.fit(data1[data1_x_bin], data1[Target])

#transform x&y to reduced features and fit new model
#alternative: can use pipeline to reduce fit and transform steps: http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
X_rfe = data1[data1_x_bin].columns.values[dtree_rfe.get_support()]
rfe_results = model_selection.cross_validate(dtree, data1[X_rfe], data1[Target], cv  = cv_split)

#print(dtree_rfe.grid_scores_)
print('AFTER DT RFE Training Shape New: ', data1[X_rfe].shape) 
print('AFTER DT RFE Training Columns New: ', X_rfe)

print("AFTER DT RFE Training w/bin score mean: {:.2f}". format(rfe_results['train_score'].mean()*100)) 
print("AFTER DT RFE Test w/bin score mean: {:.2f}". format(rfe_results['test_score'].mean()*100))
print("AFTER DT RFE Test w/bin score 3*std: +/- {:.2f}". format(rfe_results['test_score'].std()*100*3))
print('-'*10)


#tune rfe model
rfe_tune_model = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), param_grid=param_grid, scoring = 'roc_auc', cv = cv_split)
rfe_tune_model.fit(data1[X_rfe], data1[Target])

#print(rfe_tune_model.cv_results_.keys())
#print(rfe_tune_model.cv_results_['params'])
print('AFTER DT RFE Tuned Parameters: ', rfe_tune_model.best_params_)
#print(rfe_tune_model.cv_results_['mean_train_score'])
print("AFTER DT RFE Tuned Training w/bin score mean: {:.2f}". format(rfe_tune_model.cv_results_['mean_train_score'][tune_model.best_index_]*100)) 
#print(rfe_tune_model.cv_results_['mean_test_score'])
print("AFTER DT RFE Tuned Test w/bin score mean: {:.2f}". format(rfe_tune_model.cv_results_['mean_test_score'][tune_model.best_index_]*100))
print("AFTER DT RFE Tuned Test w/bin score 3*std: +/- {:.2f}". format(rfe_tune_model.cv_results_['std_test_score'][tune_model.best_index_]*100*3))
print('-'*10)

In [ ]:
#Graph MLA version of Decision Tree: http://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html
import graphviz 
dot_data = tree.export_graphviz(dtree, out_file=None, 
                                feature_names = data1_x_bin, class_names = True,
                                filled = True, rounded = True)
graph = graphviz.Source(dot_data) 
graph

## 4.8 Validación e implementación

Para comparar y elegir la mejor combinación de algoritmos de predicción, establece un análisis a través de un diagrama de correlación entre algoritmos de predicción.

In [ ]:
#compare algorithm predictions with each other, where 1 = exactly similar and 0 = exactly opposite
#there are some 1's, but enough blues and light reds to create a "super algorithm" by combining them
correlation_heatmap(MLA_predict)

Al final acaba escogiendo un `voting classifier` para poder utilizarlos todos.

In [ ]:
#why choose one model, when you can pick them all with voting classifier
#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html
#removed models w/o attribute 'predict_proba' required for vote classifier and models with a 1.0 correlation to another model
vote_est = [
    #Ensemble Methods: http://scikit-learn.org/stable/modules/ensemble.html
    ('ada', ensemble.AdaBoostClassifier()),
    ('bc', ensemble.BaggingClassifier()),
    ('etc',ensemble.ExtraTreesClassifier()),
    ('gbc', ensemble.GradientBoostingClassifier()),
    ('rfc', ensemble.RandomForestClassifier()),

    #Gaussian Processes: http://scikit-learn.org/stable/modules/gaussian_process.html#gaussian-process-classification-gpc
    ('gpc', gaussian_process.GaussianProcessClassifier()),
    
    #GLM: http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
    ('lr', linear_model.LogisticRegressionCV()),
    
    #Navies Bayes: http://scikit-learn.org/stable/modules/naive_bayes.html
    ('bnb', naive_bayes.BernoulliNB()),
    ('gnb', naive_bayes.GaussianNB()),
    
    #Nearest Neighbor: http://scikit-learn.org/stable/modules/neighbors.html
    ('knn', neighbors.KNeighborsClassifier()),
    
    #SVM: http://scikit-learn.org/stable/modules/svm.html
    ('svc', svm.SVC(probability=True)),
    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
   ('xgb', XGBClassifier())

]


#Hard Vote or majority rules
vote_hard = ensemble.VotingClassifier(estimators = vote_est , voting = 'hard')
vote_hard_cv = model_selection.cross_validate(vote_hard, data1[data1_x_bin], data1[Target], cv  = cv_split)
vote_hard.fit(data1[data1_x_bin], data1[Target])

print("Hard Voting Training w/bin score mean: {:.2f}". format(vote_hard_cv['train_score'].mean()*100)) 
print("Hard Voting Test w/bin score mean: {:.2f}". format(vote_hard_cv['test_score'].mean()*100))
print("Hard Voting Test w/bin score 3*std: +/- {:.2f}". format(vote_hard_cv['test_score'].std()*100*3))
print('-'*10)


#Soft Vote or weighted probabilities
vote_soft = ensemble.VotingClassifier(estimators = vote_est , voting = 'soft')
vote_soft_cv = model_selection.cross_validate(vote_soft, data1[data1_x_bin], data1[Target], cv  = cv_split)
vote_soft.fit(data1[data1_x_bin], data1[Target])

print("Soft Voting Training w/bin score mean: {:.2f}". format(vote_soft_cv['train_score'].mean()*100)) 
print("Soft Voting Test w/bin score mean: {:.2f}". format(vote_soft_cv['test_score'].mean()*100))
print("Soft Voting Test w/bin score 3*std: +/- {:.2f}". format(vote_soft_cv['test_score'].std()*100*3))
print('-'*10)

In [ ]:
#WARNING: Running is very computational intensive and time expensive.
#Code is written for experimental/developmental purposes and not production ready!


#Hyperparameter Tune with GridSearchCV: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
grid_n_estimator = [10, 50, 100, 300]
grid_ratio = [.1, .25, .5, .75, 1.0]
grid_learn = [.01, .03, .05, .1, .25]
grid_max_depth = [2, 4, 6, 8, 10, None]
grid_min_samples = [5, 10, .03, .05, .10]
grid_criterion = ['gini', 'entropy']
grid_bool = [True, False]
grid_seed = [0]


grid_param = [
            [{
            #AdaBoostClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
            'n_estimators': grid_n_estimator, #default=50
            'learning_rate': grid_learn, #default=1
            #'algorithm': ['SAMME', 'SAMME.R'], #default=’SAMME.R
            'random_state': grid_seed
            }],
       
    
            [{
            #BaggingClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier
            'n_estimators': grid_n_estimator, #default=10
            'max_samples': grid_ratio, #default=1.0
            'random_state': grid_seed
             }],

    
            [{
            #ExtraTreesClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier
            'n_estimators': grid_n_estimator, #default=10
            'criterion': grid_criterion, #default=”gini”
            'max_depth': grid_max_depth, #default=None
            'random_state': grid_seed
             }],


            [{
            #GradientBoostingClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier
            #'loss': ['deviance', 'exponential'], #default=’deviance’
            'learning_rate': [.05], #default=0.1 -- 12/31/17 set to reduce runtime -- The best parameter for GradientBoostingClassifier is {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300, 'random_state': 0} with a runtime of 264.45 seconds.
            'n_estimators': [300], #default=100 -- 12/31/17 set to reduce runtime -- The best parameter for GradientBoostingClassifier is {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300, 'random_state': 0} with a runtime of 264.45 seconds.
            #'criterion': ['friedman_mse', 'mse', 'mae'], #default=”friedman_mse”
            'max_depth': grid_max_depth, #default=3   
            'random_state': grid_seed
             }],

    
            [{
            #RandomForestClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
            'n_estimators': grid_n_estimator, #default=10
            'criterion': grid_criterion, #default=”gini”
            'max_depth': grid_max_depth, #default=None
            'oob_score': [True], #default=False -- 12/31/17 set to reduce runtime -- The best parameter for RandomForestClassifier is {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 100, 'oob_score': True, 'random_state': 0} with a runtime of 146.35 seconds.
            'random_state': grid_seed
             }],
    
            [{    
            #GaussianProcessClassifier
            'max_iter_predict': grid_n_estimator, #default: 100
            'random_state': grid_seed
            }],
        
    
            [{
            #LogisticRegressionCV - http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV
            'fit_intercept': grid_bool, #default: True
            #'penalty': ['l1','l2'],
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], #default: lbfgs
            'random_state': grid_seed
             }],
            
    
            [{
            #BernoulliNB - http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB
            'alpha': grid_ratio, #default: 1.0
             }],
    
    
            #GaussianNB - 
            [{}],
    
            [{
            #KNeighborsClassifier - http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
            'n_neighbors': [1,2,3,4,5,6,7], #default: 5
            'weights': ['uniform', 'distance'], #default = ‘uniform’
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
            }],
            
    
            [{
            #SVC - http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
            #http://blog.hackerearth.com/simple-tutorial-svm-parameter-tuning-python-r
            #'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [1,2,3,4,5], #default=1.0
            'gamma': grid_ratio, #edfault: auto
            'decision_function_shape': ['ovo', 'ovr'], #default:ovr
            'probability': [True],
            'random_state': grid_seed
             }],

    
            [{
            #XGBClassifier - http://xgboost.readthedocs.io/en/latest/parameter.html
            'learning_rate': grid_learn, #default: .3
            'max_depth': [1,2,4,6,8,10], #default 2
            'n_estimators': grid_n_estimator, 
            'seed': grid_seed  
             }]   
        ]



start_total = time.perf_counter() #https://docs.python.org/3/library/time.html#time.perf_counter
for clf, param in zip (vote_est, grid_param): #https://docs.python.org/3/library/functions.html#zip

    #print(clf[1]) #vote_est is a list of tuples, index 0 is the name and index 1 is the algorithm
    #print(param)
    
    
    start = time.perf_counter()        
    best_search = model_selection.GridSearchCV(estimator = clf[1], param_grid = param, cv = cv_split, scoring = 'roc_auc')
    best_search.fit(data1[data1_x_bin], data1[Target])
    run = time.perf_counter() - start

    best_param = best_search.best_params_
    print('The best parameter for {} is {} with a runtime of {:.2f} seconds.'.format(clf[1].__class__.__name__, best_param, run))
    clf[1].set_params(**best_param) 


run_total = time.perf_counter() - start_total
print('Total optimization time was {:.2f} minutes.'.format(run_total/60))

print('-'*10)

In [ ]:
#Hard Vote or majority rules w/Tuned Hyperparameters
grid_hard = ensemble.VotingClassifier(estimators = vote_est , voting = 'hard')
grid_hard_cv = model_selection.cross_validate(grid_hard, data1[data1_x_bin], data1[Target], cv  = cv_split)
grid_hard.fit(data1[data1_x_bin], data1[Target])

print("Hard Voting w/Tuned Hyperparameters Training w/bin score mean: {:.2f}". format(grid_hard_cv['train_score'].mean()*100)) 
print("Hard Voting w/Tuned Hyperparameters Test w/bin score mean: {:.2f}". format(grid_hard_cv['test_score'].mean()*100))
print("Hard Voting w/Tuned Hyperparameters Test w/bin score 3*std: +/- {:.2f}". format(grid_hard_cv['test_score'].std()*100*3))
print('-'*10)

#Soft Vote or weighted probabilities w/Tuned Hyperparameters
grid_soft = ensemble.VotingClassifier(estimators = vote_est , voting = 'soft')
grid_soft_cv = model_selection.cross_validate(grid_soft, data1[data1_x_bin], data1[Target], cv  = cv_split)
grid_soft.fit(data1[data1_x_bin], data1[Target])

print("Soft Voting w/Tuned Hyperparameters Training w/bin score mean: {:.2f}". format(grid_soft_cv['train_score'].mean()*100)) 
print("Soft Voting w/Tuned Hyperparameters Test w/bin score mean: {:.2f}". format(grid_soft_cv['test_score'].mean()*100))
print("Soft Voting w/Tuned Hyperparameters Test w/bin score 3*std: +/- {:.2f}". format(grid_soft_cv['test_score'].std()*100*3))
print('-'*10)


#12/31/17 tuned with data1_x_bin
#The best parameter for AdaBoostClassifier is {'learning_rate': 0.1, 'n_estimators': 300, 'random_state': 0} with a runtime of 33.39 seconds.
#The best parameter for BaggingClassifier is {'max_samples': 0.25, 'n_estimators': 300, 'random_state': 0} with a runtime of 30.28 seconds.
#The best parameter for ExtraTreesClassifier is {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 100, 'random_state': 0} with a runtime of 64.76 seconds.
#The best parameter for GradientBoostingClassifier is {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 300, 'random_state': 0} with a runtime of 34.35 seconds.
#The best parameter for RandomForestClassifier is {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 100, 'oob_score': True, 'random_state': 0} with a runtime of 76.32 seconds.
#The best parameter for GaussianProcessClassifier is {'max_iter_predict': 10, 'random_state': 0} with a runtime of 6.01 seconds.
#The best parameter for LogisticRegressionCV is {'fit_intercept': True, 'random_state': 0, 'solver': 'liblinear'} with a runtime of 8.04 seconds.
#The best parameter for BernoulliNB is {'alpha': 0.1} with a runtime of 0.19 seconds.
#The best parameter for GaussianNB is {} with a runtime of 0.04 seconds.
#The best parameter for KNeighborsClassifier is {'algorithm': 'brute', 'n_neighbors': 7, 'weights': 'uniform'} with a runtime of 4.84 seconds.
#The best parameter for SVC is {'C': 2, 'decision_function_shape': 'ovo', 'gamma': 0.1, 'probability': True, 'random_state': 0} with a runtime of 29.39 seconds.
#The best parameter for XGBClassifier is {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300, 'seed': 0} with a runtime of 46.23 seconds.
#Total optimization time was 5.56 minutes.

De esta manera consigue lo que quería, a través de estos métodos, incrementar el score llegando hasta un 85.22%

Comparando este trabajo con el nuestro, resalta lo laborioso y más desarrollado que está, llegando incluso a establecer análisis muy profundos para los clasificadores y las validaciones, además de las preguntas previas.

Otro punto también destacable se encuentra en el preprocesamiento, ya que nosostros no llegamos a esablecer relaciones entre variables, simplemente realizamos una limpieza.

Otro dato interesante es la validación cruzada de 10 repeticiones con 3 carpetas test y 6 carpetas train, mientras que nosotros siempre nos hemos limitado a unas 10 repeticiones con 1 carpeta test y 5 carpetas train, hubiera estado también interesante este tipo de combinaciones.

En general este kernel de kaggle es muy completo y superior al trabajo que nosotros hemos realizado, ya que va más allá en muchas de las cosas que nosotros realizamos, lo que la convierte en una libreta muy interesante para estudiar detenidamente.